In [19]:
import torch
import torch.nn as nn



class Model(nn.Module):
    def __init__(self,num_features):
        super().__init__()
        self.linear1 = nn.Linear(num_features,5)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(5,1)
        self.sigmoid = nn.Sigmoid()
    def forward(self,features):
        out = self.linear1(features)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.sigmoid(out)
        return out

    def loss_function(self,y_pred,y_true):
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)
        loss = -(y_true * torch.log(y_pred)+(1-y_true)*torch.log(1-y_pred)).mean()
        return loss


In [20]:
from sklearn.datasets import load_breast_cancer
lbc = load_breast_cancer()
X = lbc['data']
y = lbc['target']
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
Y_train = torch.from_numpy(Y_train).float()
Y_test = torch.from_numpy(Y_test).float()
X_train.to(device=torch.device('cuda'))
X_test.to(device=torch.device('cuda'))
Y_train.to(device=torch.device('cuda'))
Y_test.to(device=torch.device('cuda'))


tensor([1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1.,
        1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
        0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
        1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0.,
        1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 1., 1., 0.], device='cuda:0')

In [21]:
learning_rate = 0.05
epochs = 100

In [22]:
loss_function = nn.BCELoss()

In [23]:
model = Model(X_train.shape[1])
optimiser = torch.optim.SGD(model.parameters(), lr=learning_rate)
for epoch in range(epochs):
    y_pred = model.forward(X_train)
    loss = loss_function(y_pred, Y_train.view(-1,1))
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    print(f'epoch: {epoch}, loss: {loss}')


epoch: 0, loss: 0.7168493270874023
epoch: 1, loss: 0.7014760971069336
epoch: 2, loss: 0.6870385408401489
epoch: 3, loss: 0.6734253764152527
epoch: 4, loss: 0.6607495546340942
epoch: 5, loss: 0.6490194201469421
epoch: 6, loss: 0.6375851631164551
epoch: 7, loss: 0.6266297698020935
epoch: 8, loss: 0.6161724925041199
epoch: 9, loss: 0.6061422824859619
epoch: 10, loss: 0.5964657068252563
epoch: 11, loss: 0.5870941281318665
epoch: 12, loss: 0.5779328942298889
epoch: 13, loss: 0.568848729133606
epoch: 14, loss: 0.5598723888397217
epoch: 15, loss: 0.5511026382446289
epoch: 16, loss: 0.5425317287445068
epoch: 17, loss: 0.5341066122055054
epoch: 18, loss: 0.5257436037063599
epoch: 19, loss: 0.5174406170845032
epoch: 20, loss: 0.5092602372169495
epoch: 21, loss: 0.5011597871780396
epoch: 22, loss: 0.49323418736457825
epoch: 23, loss: 0.4854499101638794
epoch: 24, loss: 0.4777540862560272
epoch: 25, loss: 0.47018226981163025
epoch: 26, loss: 0.4628104269504547
epoch: 27, loss: 0.455571711063385
ep

In [37]:
with torch.no_grad():
    y_pred = model.forward(X_test)
    y_pred = (y_pred>0.00000001).float()
    accuracy = (y_pred == Y_test).float().mean()
    print(f'test accuracy: {accuracy}')

test accuracy: 0.6228070259094238
